In [1]:
from google.colab import userdata
openai_key = userdata.get('OPENAI')
google_key = userdata.get('GOOGLE')

import os
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["GOOGLE_API_KEY"] = google_key

ModuleNotFoundError: No module named 'google.colab'

#Semi-structured RAG :

###Extract elements (table, text, images) using unstructured :

In [2]:
!apt-get  install poppler-utils tesseract-ocr
!pip install --upgrade nltk
!pip install -q unstructured["all-docs"]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 5,002 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 5,002 kB in 0s (14.4 MB/s)
Selecting previously unselected package poppl

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
from unstructured.partition.pdf import partition_pdf

In [5]:
pdf_elements = partition_pdf(
    filename=  "/content/tsla (1).pdf",
    extract_images_in_pdf=False,
    infer_table_structure=True,

    ##Chunking
    chunking_strategy="by_title",
    max_characters=4000,                #maximun no. of characters in chunks
    new_after_n_chars = 3800,           #200 chars overlap
    combine_text_under_n_chars = 3000,  #
)

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
different_element = []

# {"type": "table", "text": "gfhjguhk"}

for element in pdf_elements:
  if "unstructured.documents.elements.Table" in str(type(element)):
    different_element.append({"type" : "table", "text" : str(element)})
  elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
    different_element.append({"type" : "text", "text" : str(element)})

In [7]:
table = [tt for tt in different_element if tt["type"] == "table"]
text = [tt for tt in different_element if tt["type"] == "text"]

In [8]:
len(table), len(text)

(13, 25)

###Create Summary :

In [ ]:
!pip install langchain langchain-community openai tiktoken chromadb

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema import HumanMessage, SystemMessage

In [11]:
prompt_template = """Your task is to create summaries from table and text.
give clear and concise summary of give table or text chunk. chunks : {chunk}
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

model = ChatOpenAI(model="gpt-4o")
parser = StrOutputParser()

summary_chain = prompt | model | parser

<ipython-input-11-27f00da3a268>:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(model="gpt-4o")


In [13]:
table[0]

{'type': 'table',
 'text': 'Title of each class Trading Symbol(s) Name of each exchange on which registered Common stock TSLA The Nasdaq Global Select Market'}

In [12]:
summary_chain.invoke(table[0]["text"])


"Tesla, Inc.'s common stock, trading under the symbol TSLA, is listed on The Nasdaq Global Select Market."

In [14]:
table_text = [tt["text"] for tt in table]
text_text = [t["text"] for t in text]

In [15]:
table_summary = summary_chain.batch(table_text)
text_summary = summary_chain.batch(text_text)

###Retrievers :

In [16]:
from langchain.retrievers import MultiVectorRetriever
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore.document import Document
import uuid

In [17]:
embedding = OpenAIEmbeddings()

db = Chroma(collection_name="summary", embedding_function=embedding)

<ipython-input-17-90ac6e31fc32>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
<ipython-input-17-90ac6e31fc32>:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(collection_name="summary", embedding_function=embedding)


In [18]:
from langchain.storage import InMemoryStore
store = InMemoryStore()

In [19]:
retriever = MultiVectorRetriever(
    vectorstore=db,  # to store summary embeddings
    docstore=store,  # to store  chunks
    id_key="doc_id", # to map chunks to its respective summary
)

In [20]:
table_ids = [str(uuid.uuid4()) for _ in range(len(table_summary))]

summary_table_doc = []
for ind, summary in enumerate(table_summary):
  doc = Document(page_content=summary, metadata={"doc_id" : table_ids[ind]})
  # db.add_documents([doc])
  summary_table_doc.append(doc)

retriever.docstore.mset(list(zip(table_ids, table_text)))
retriever.vectorstore.add_documents(summary_table_doc)

['78f760dd-1a0b-46e6-82a9-925df0fc09c8',
 '0b561ea8-4952-41e1-adaa-0d9889aca059',
 '3cc7c2ca-edad-407b-b21d-5dfde208fdb3',
 '4eb0b125-e095-4fcb-893e-44ce1793bb4a',
 '56c8d1d3-3db1-4f9b-8769-8b2a00744c48',
 'db4dd5dd-ed3e-45a3-85f7-eb871e428664',
 '6e584006-e024-4624-8eb8-b74b9a728ec9',
 '37140514-7009-4fea-9ccb-c46981a7e230',
 'c8a5d84e-801d-4fe4-9108-6717fc995223',
 'd8c36747-1b23-482e-88c0-a18cb2b61d51',
 '1673d7ff-73dc-4702-bb3c-9f3d20452cbb',
 '80c1d8e0-07b5-4cea-af39-9cdd0959b957',
 '9be8e5c1-30bf-43c3-af72-5fca327a4e9c']

In [21]:
text_ids = [str(uuid.uuid4()) for _ in range(len(text_summary))]

summary_text_doc = []
for ind, summary in enumerate(text_summary):
  doc = Document(page_content=summary, metadata={"doc_id" : text_ids[ind]})
  # db.add_documents([doc])
  summary_text_doc.append(doc)

retriever.docstore.mset(list(zip(text_ids, text_text)))
retriever.vectorstore.add_documents(summary_text_doc)

['530164e8-aef9-45b6-b0f3-5d2955bea5b6',
 'd2696b60-b157-4e27-a026-fc40e317c129',
 'ce5362a8-5c80-4805-a756-f4889ff506f5',
 '7b4137d2-68db-4152-8312-57380ee30c00',
 'e4ecb96b-dbc1-4ca4-82a9-2e558a758112',
 '4a23051c-9b4b-4f49-a049-62eb23047065',
 '8ead6867-17fa-423d-8d2a-54c2836e330d',
 'c0dc0e4e-67f0-4bfd-b26f-385f34c50491',
 '65106ed6-c876-4131-a601-0503f1e1713a',
 'ea1a6a4c-8387-4f00-825f-a947f7c6bdb5',
 'bd24582d-8fd2-4152-acfd-8abee660e84f',
 'd921810d-88b4-4f07-977a-1eefab793a33',
 '1fa401ec-0f9b-4c67-9a43-c1a3dea845b4',
 '5d70c5b6-346a-4020-8cdb-c7d6e2635e95',
 'cdaf15d1-5a95-4884-883d-195f5936a77f',
 'd026f0ef-b7ee-42c6-980c-4aa3e3b0dcc8',
 'aee38053-d5cb-4547-a116-c3d5cacb2c30',
 'fd212acd-6bca-462a-8960-e0d05f01b2a3',
 'c6d02b46-b356-4e90-9982-b75ca1ab40ac',
 '70122c8b-22c1-42d3-a26e-823f4e4550ee',
 'b8be3405-66f9-4f7f-a29a-8950684e3012',
 '3659e05c-31f1-4909-ab10-500dd025c722',
 '2c80bf5c-44b1-4243-af69-8ebea973bffe',
 'bc286eef-0fd3-47cd-929c-d2a869bf3661',
 'c927134f-ba84-

###Chain :

In [28]:
from langchain.schema.runnable import RunnablePassthrough

In [29]:
prompt_template = """Anaswer the question based on given context. context can be table or text.
If answer is not present in context just give "I don't know" withot any explaination.
question : {question}
context : {context}
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

model = ChatOpenAI(model="gpt-4o")
parser = StrOutputParser()

chain = {"context" : retriever, "question": RunnablePassthrough() } | prompt | model | parser

# RunnablePassthrough  : take this variable from, what is passed in .invoke
# RunnableLamda :

In [26]:
# retriever.invoke("how much is tesla revenue in automative sales in 2021?")

In [30]:
chain.invoke("how much is tesla revenue in automative sales in 2021?")

'$44,125 million'

In [31]:
chain.invoke("who is virat kohli?")

"I don't know."

##Images in RAG :

In [33]:
pdf_elements = partition_pdf(
    filename=  "/content/tsla (1)_merged.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,

    ##Chunking
    chunking_strategy="by_title",
    max_characters=4000,                #maximun no. of characters in chunks
    new_after_n_chars = 3800,           #200 chars overlap
    combine_text_under_n_chars = 3000,  #
)

In [34]:
import base64

In [ ]:
1. here we are converting our images in string, then we are storing them in inmemorystore. in  case of table and text we can store them directly in inmemorystore, but for images we have to convert them into string.
2. creating image_string
    def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    binary = image_file.read()

    asci_string = base64.b64encode(binary)  ##asci
    image_string = asci_string.decode("utf-8")

    return image_string
3.  summarizing image

    def summarize_image(image_url):
  prompt = """Your task is to create summaries from images, these summaries will
  be used to embedd and retrieve images.
  """
  message = HumanMessage(content=[
    {"type": "text", "text" : prompt},
    {"type":"image_url", "image_url":{"url": image_url}}
  ])
  summary = llm.invoke([message]).content

  return summary

4. image_strings = []

image_summary = []

folder = "/content/figures"
for img in os.listdir(folder):
  img_path = os.path.join(folder, img)
  print(img_path)
  image_strings.append(encode_image(img_path))
  image_summary.append(summarize_image(img_path))
  
5. After creating the image_string and image_summary, we have to store image_string in inmemorystore(docstore) and vectorestore(vector database)


image_ids = [str(uuid.uuid4()) for _ in range(len(image_summary))]

summary_image_doc = []
for ind, summary in enumerate(image_summary):
  doc = Document(page_content=summary, metadata={"doc_id" : image_ids[ind]})
  # db.add_documents([doc])
  summary_image_doc.append(doc)

retriever.docstore.mset(list(zip(image_ids, image_strings)))
retriever.vectorstore.add_documents(summary_image_doc)


# Create RAG chain
import io
import re

from IPython.display import HTML, display
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from PIL import Image


def looks_like_base64(sb):
    """Check if the string looks like base64"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


def is_image_data(b64data):
    """
    Check if the base64 data is an image by looking at the start of the data
    """
    image_signatures = {
        b"\xff\xd8\xff": "jpg",
        b"\x89\x50\x4e\x47\x0d\x0a\x1a\x0a": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False


def resize_base64_image(base64_string, size=(128, 128)):
    """
    Resize an image encoded as a Base64 string
    """
    # Decode the Base64 string
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Resize the image
    resized_img = img.resize(size, Image.LANCZOS)

    # Save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # Encode the resized image to Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def split_image_text_types(docs):
    """
    Split base64-encoded images and texts
    """
    b64_images = []
    texts = []
    for doc in docs:
        # Check if the document is of type Document and extract page_content if so
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc = resize_base64_image(doc, size=(1300, 600))
            b64_images.append(doc)
        else:
            texts.append(doc)
    return {"images": b64_images, "texts": texts}


def img_prompt_func(data_dict):
    """
    Join the context into a single string
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    # Adding the text for analysis
    text_message = {
        "type": "text",
        "text": (
            f"""You are financial analyst.\n
            You will be given a mixed of text, tables, and image(s) usually of charts or graphs.\n
            Use this information to provide answer related to the user question. \n
            User-provided question: {data_dict['question']}\n\n
            Text and / or tables:\n
            {formatted_texts}"""
        ),
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]


def multi_modal_rag_chain(retriever):
    """
    Multi-modal RAG chain
    """

    # Multi-modal LLM
    # model = ChatOpenAI(temperature=0, model="gpt-4-vision-preview", max_tokens=1024)

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | llm
        | StrOutputParser()
    )

    return chain


6. After this we need to create RAG chain. We have stored images in string format and we cannot pass them to the model in 
string format. So we will use rannablelambda. We will append our images in the previous retriever. So we can retrieve data 
from table, text and images.

chain_multimodal_rag = multi_modal_rag_chain(retriever)

chain_multimodal_rag.invoke("query")



In [36]:
with open("/content/rag.png", "rb") as image_file:
    binary = image_file.read()

    asci_string = base64.b64encode(binary)  ##asci
    image_string = asci_string.decode("utf-8")

In [ ]:
image_string

In [51]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    binary = image_file.read()

    asci_string = base64.b64encode(binary)  ##asci
    image_string = asci_string.decode("utf-8")

    return image_string


###Image Summary :

In [ ]:
!pip install -U langchain-google-genai

In [41]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [42]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [43]:
from langchain.schema.messages import HumanMessage

In [47]:
message = HumanMessage(content=[
    {"type": "text", "text" : "summarize given image"},
    {"type":"image_url", "image_url":{"url": "/content/rag.png"}}
])

In [48]:
llm.invoke([message]).content

'The image shows a system that uses a large language model (LLM) to answer questions based on a set of documents. The user provides a query, which is then processed by an embeddings model and stored in a vectorstore. The system then retrieves relevant documents from the vectorstore based on the query, and uses them to create a context query prompt for the LLM. The LLM then generates an output based on the prompt, which is then presented to the user. This process allows the system to provide accurate and relevant answers to user queries based on the information contained in the documents.'

In [49]:
def summarize_image(image_url):
  prompt = """Your task is to create summaries from images, these summaries will
  be used to embedd and retrieve images.
  """
  message = HumanMessage(content=[
    {"type": "text", "text" : prompt},
    {"type":"image_url", "image_url":{"url": image_url}}
  ])
  summary = llm.invoke([message]).content

  return summary

In [50]:
summarize_image("/content/rag.png")

'A user asks a question. The question is sent to an embedding model and then the embeddings are stored in a vectorstore. The vectorstore is then used to retrieve relevant documents. The retrieved documents are then used to create a context query prompt which is sent to an LLM. The LLM generates an output.'

In [54]:
image_strings = []

image_summary = []

folder = "/content/figures"
for img in os.listdir(folder):
  # print(img)
  img_path = os.path.join(folder, img)
  print(img_path)
  image_strings.append(encode_image(img_path))
  image_summary.append(summarize_image(img_path))

/content/figures/figure-2-4.jpg
/content/figures/figure-2-6.jpg
/content/figures/figure-3-8.jpg
/content/figures/figure-4-11.jpg
/content/figures/figure-2-5.jpg
/content/figures/figure-5-13.jpg
/content/figures/figure-3-9.jpg
/content/figures/figure-4-12.jpg
/content/figures/figure-2-3.jpg
/content/figures/figure-1-1.jpg
/content/figures/figure-3-7.jpg
/content/figures/figure-4-10.jpg
/content/figures/figure-1-2.jpg


In [55]:
image_ids = [str(uuid.uuid4()) for _ in range(len(image_summary))]

summary_image_doc = []
for ind, summary in enumerate(image_summary):
  doc = Document(page_content=summary, metadata={"doc_id" : image_ids[ind]})
  # db.add_documents([doc])
  summary_image_doc.append(doc)

retriever.docstore.mset(list(zip(image_ids, image_strings)))
retriever.vectorstore.add_documents(summary_image_doc)

['543096c0-8fcd-4020-a7f8-e7604796665a',
 '8f04a78a-c731-42f8-9bc9-52182ae8b6e2',
 '330cbb6e-b338-4f7c-9fe0-608e703acaab',
 '17d37b5d-1904-425e-b596-a604d0c0e2c5',
 'bd783920-aad9-4be1-8311-d65e995977dc',
 'de5f5b13-64f2-4074-8872-32ee6bfc9ecc',
 'd09a5966-15e3-4730-9e37-4d2f90d88f93',
 '79548b7e-d173-4c9e-a25b-9fe37ffa35fc',
 '06acc4f5-5894-49c2-8cd5-8ebddf632cc8',
 '64a4e8ec-e78d-4e5b-887c-7e78248f2623',
 '33ef7cad-2d2b-4981-b8e9-205fab1c6f2f',
 '97f4fbe0-708a-4a9f-bfd0-4ffe14280301',
 'f97751cd-895d-4b29-bfc1-b0c5b8684d10']

In [57]:
# Create RAG chain
import io
import re

from IPython.display import HTML, display
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from PIL import Image


def looks_like_base64(sb):
    """Check if the string looks like base64"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


def is_image_data(b64data):
    """
    Check if the base64 data is an image by looking at the start of the data
    """
    image_signatures = {
        b"\xff\xd8\xff": "jpg",
        b"\x89\x50\x4e\x47\x0d\x0a\x1a\x0a": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False


def resize_base64_image(base64_string, size=(128, 128)):
    """
    Resize an image encoded as a Base64 string
    """
    # Decode the Base64 string
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Resize the image
    resized_img = img.resize(size, Image.LANCZOS)

    # Save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # Encode the resized image to Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def split_image_text_types(docs):
    """
    Split base64-encoded images and texts
    """
    b64_images = []
    texts = []
    for doc in docs:
        # Check if the document is of type Document and extract page_content if so
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc = resize_base64_image(doc, size=(1300, 600))
            b64_images.append(doc)
        else:
            texts.append(doc)
    return {"images": b64_images, "texts": texts}


def img_prompt_func(data_dict):
    """
    Join the context into a single string
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    # Adding the text for analysis
    text_message = {
        "type": "text",
        "text": (
            f"""You are financial analyst.\n
            You will be given a mixed of text, tables, and image(s) usually of charts or graphs.\n
            Use this information to provide answer related to the user question. \n
            User-provided question: {data_dict['question']}\n\n
            Text and / or tables:\n
            {formatted_texts}"""
        ),
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]


def multi_modal_rag_chain(retriever):
    """
    Multi-modal RAG chain
    """

    # Multi-modal LLM
    # model = ChatOpenAI(temperature=0, model="gpt-4-vision-preview", max_tokens=1024)

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | llm
        | StrOutputParser()
    )

    return chain


In [58]:
# RunnableLambda : run function in chain

In [59]:

chain_multimodal_rag = multi_modal_rag_chain(retriever)

#  chain
chain_multimodal_rag.invoke("How much was tesla revenue in 2022?")

"Tesla's revenue in 2022 was **$81,462 million**. \n"

In [61]:
chain_multimodal_rag.invoke("explain how taj deccan hyderabad looks like?")

'Taj Deccan Hyderabad is a white building with a large, central section that has three balconies. There are trees and greenery in front of the building, and a driveway leads up to the entrance.'

In [62]:
chain_multimodal_rag.invoke("how much is tesla revenue in automative sales in 2021?")

"Tesla's automotive sales revenue in 2021 was **$44,125 million**. \n"

In [ ]:
# "llava-hf/llava-1.5-7b-hf" ==>